## OpenAI API Simple Prompt Engineering Examples Using ChatGPT

Welcome! This notebook is meant to show some simple prompt engineering examples for those just getting started in the field. We will look at four common tasks associated with typical prompt engineering job requirements to get you on the road to becoming an effective developer:

- Content Generation
- Content Summariziation
- Language Translation
- Content Automation

There are a couple of things to consider as we explore these concepts:
First, we are limiting ourselves to exclusively using OpenAI's API even though there might be better alternatives out there to keep things easy to impelement. 
Second, You will need an OpenAI API key to do these lessons. They give you a $5 credit when you sign up but IT CAN COST MONEY if you aren't careful. 
Just be aware of your usage patterns and you will be fine. You can sign up for your API access here: https://platform.openai.com/ 

Let's get started!


### Content Generation
Assume we work for a book company and they are looking at expanding into the children book market. They don't want to invest a lot of money in authors before they have tested the waters a bit. That's where we come in; our first task is to write a prompt to create a kids short story. The customer wants us to generate a story and make sure the AI outputs an HTML table with the names of the characters from the story and their role in the story so they can put it up on their website. 

In [6]:
# import our packages
import os
import openai

# grab our API key from our environment variables
# NOTE: if you don't have an environment variable set, you can just paste your key in directly by modifying the line below and setting your openai_key to your key
# like this: openai_key = "your key goes here"
openai_key = os.getenv('OPENAI_KEY')

openai.api_key = openai_key

# craft our prompt - this is the text that the AI will use to generate the content
# In this example we use the persona feature to give the AI a little bit of context about the type of content we want it to generate
# Specifically we want it to be a children story teller generate content in the style of Hans Christian Andersen
# This sets the tone and style of the content we want to generate
# Next, we give the AI some direction about the type of content we want it to generate to set the "boundaries" of the content - these are typically given to us by the client
#
# Now we deal with the output
# We clearly indicate where the story should go and where the character and role table should go to make it easy to parse out the content
prompt = """
Act as a children story teller in the style of Hans Christian Andersen using the following process:
1. Make the story no more than 1000 words.
2. Create a story that begins with an opening scene that captures the reader's imagination. 
3. Set the scene by providing a detailed description of the setting and atmosphere, including the time of day, the weather, and any other relevant details that help to immerse the reader in the story. 
4. Introduce the minor characters in the story, providing enough detail about each character to make them interesting and memorable. 
5. Introduce the main character of the story, describing their physical appearance in detail and giving some insight into their personality and motivations. 
6. Make sure that the story has a clear plot and a satisfying resolution, and that the themes explored in the story are appropriate for a young audience. 
7. Finally, be sure to use vivid language and imagery to bring the story to life and engage the reader's imagination. 

The story can be about anything you want. Be creative!

When you have given me the story, then produce a HTML table with the names of the characters from the story and their role in the story.

Story: <story goes here>

Characters and Roles: <HTML table goes here>

"""
# Now we use the completion.create() method to generate the content and things really get interesting. See my comments on each of the parameters below
response = openai.Completion.create(
    # This is the model we want to use to generate the content - in this case we are using a ChatGPT 3.5 model because it is cheaper to use than the GPT-4 model
    # Useful for prototyping and testing then using the GPT-4 model for later iterations
    # At the time of this writing, for input the GPT-3.5 model costs $0.0015 per 1000 tokens and the GPT-4 model costs $0.03 per 1000 tokens; for output the 
    # GPT-3.5 model costs $0.002 per 1000 tokens and the GPT-4 model costs $0.06 per 1000 tokens
    # Quite a cost differential between the two models so start cheap to prototype and test and then use the GPT-4 model for later on
    model="text-davinci-003",  

    # This is the prompt we created above
    prompt=prompt,  

    # This is the temperature parameter - it controls the randomness of the output or, put another way, how "creative" the AI is
    # The number can be between 0 and 1 - the closer to 0 the less "creative" the AI is and as the number approaches 1 the more "creative" the AI is
    # Since we are creating children's stories we want the AI to be more creative so we set the temperature to 0.7
    # If we were creating something like a legal document we would want the AI to be less creative so we would set the temperature to 0.2 for example
    temperature=0.7,  

    # This is the max_tokens parameter - it controls the length of the output
    # To quote OpenAI: "You can think of tokens as pieces of words, where 1,000 tokens is about 750 words."
    # our prompt is 1000 words so we set the max_tokens to 2000 to give the AI enough room to generate the content
    # But small enough to keep the cost down
    max_tokens=2000,  

    # This is the top_p parameter - it controls the range of words chosen for the output
    # The values are from 0 to 1 - the closer to 0 the less diverse the output and the closer to 1 the more diverse the output
    # For example, if we set top_p to 0.5 then the AI will only use the top 50% of the most likely words
    # But, if we set top_p to 1.0 then the AI will use all of the words
    # Since we are creating children's stories we want the AI to be more diverse so we set the top_p to 1.0
    # If we were creating something like a legal document we would want the AI to be less diverse so we would set the top_p to 0.5 or less, for example
    top_p=1,  

    # This is the frequency_penalty parameter - this parameter is used to discourage the model from repeating the same words or phrases too frequently within the generated text.
    # A higher frequency_penalty value will result in the model being more conservative in its use of repeated words. 
    # The values are from -2.0 to 2.0 - the closer to -2.0 the more likely the AI will repeat the same words and the closer to 2.0 the less likely the AI will repeat the same words
    # Typical setting for this parameter is 0.0 or to 1 for eliminating repetition in output. We will set ours to 0.5
    frequency_penalty=0.5,  

    # This is the presence_penalty parameter - this parameter is used to encourage the model to include a diverse range of words in the generated text. 
    # A higher presence_penalty value will result in the model being more likely to generate words that have not yet been included in the generated text.
    # The values are from -2.0 to 2.0 - the closer to -2.0 the more likely the AI will repeat the same words and the closer to 2.0 the more likely to include words not used before
    # As with the frequency_penalty parameter, typical setting for this parameter is 0.0 or to 1 for eliminating repetition in output. We will set ours to 0.5
    presence_penalty=0.5  
)

# Now we grab the response text from the response object output from the completion.create() method
# Notice the choices[0].text - this is because the completion.create() method can return multiple choices
# Which can be useful for some applications but for our purposes we only want the first choice
# For example, if a human were taking the output and choosing the best response then we would want to return all of the choices
response_text = response.choices[0].text

# Print the response text to the console
print(response_text)

# Save the response to a text file for later use
with open("ContentGeneration.txt", "w") as file:
    file.write(response_text)




Once upon a time, in a far away kingdom there lived a young princess named Aurora. She was known for her beauty and kindness, and everyone in the kingdom loved her dearly. Despite being the daughter of a powerful king, Aurora had a gentle soul and always showed compassion to those around her. 

One day, the king and queen decided to throw a grand banquet to celebrate their daughter's birthday. The entire kingdom was invited, but there were two special guests who had been invited – two fairies from the magical realm beyond the castle walls. 

The first fairy was Flora, an old wise woman with magical powers that could make plants grow with just one touch of her staff. The second fairy was Fauna, a mischievous sprite who delighted in causing trouble among mortals. Despite their differences, they both shared one thing - they were both devoted to Aurora and wanted nothing more than to make her happy on her special day. 

When the feast began, all eyes were on Aurora as she entered the room

### Content Summarization
Now let's assume that, based on the story created by the AI on the previous task, they want a summary for their marketing department. The customer has asked us to write a formal and informal version of the story that will be given to marketing later on. Let's do it!

In [10]:
# Since we have already imported our packages we don't need to do it again here
# We also don't need to grab our API key again since we already did that above as well


# create our story variable to hold the story text we generated before
story = response_text.split('Characters and Roles:')[0].strip()

# Craft our formal summary prompt using a paragraph instead of a list of rules
# Notice how we start with a persona "expert in summarizing stories" and then we give the AI a little bit of context about the type of content we want it to generate
# Specifically we want it to be a formal summary written in a professional tone suitable for publication
# The more context we give the AI the better the output will be
prompt_formal = f"""
As an expert in summarizing stories, your task is to provide a concise, yet comprehensive, summary of the text I provide you. Analyze each sentence and paragraph carefully to extract key information or arguments. Your final summary should be written in a professional tone, suitable for publication. Keep it detailed but within a 100-word limit.

{story}

Formal Summary: <summary goes here>
"""

response_formal = openai.Completion.create(
  model="text-davinci-003",  
  prompt=prompt_formal,  

  # Since we are creating a formal summary we want the AI to be less creative so we set the temperature to 0.3
  temperature=0.3,  
  max_tokens=200,  

  # mid-level diversity setting to strike a balance between repetition and creativity
  top_p=0.2,  

  # use the same frequency and presence penalty settings as before
  frequency_penalty=0.5,
  presence_penalty=0.5
)

formal_summary = response_formal.choices[0].text.strip().replace("Formal Summary:", "").strip()

print("Formal Summary:\n", formal_summary)


# Create informal summary
prompt_informal = f"""
Hey there, expert story summarizer! I've got a story for you. Your job? Sum it up like you're chatting with a close friend. Go through it line by line, grab all the cool stuff or important points, and spin it into a fun summary. Keep it light and entertaining but don't go over 100 words, alright?

{story}

Informal Summary: <summary goes here>
"""

response_informal = openai.Completion.create(
  model="text-davinci-003",  
  prompt=prompt_informal,  
  temperature=0.9,  
  max_tokens=200,  
  top_p=1.0,  
  frequency_penalty=0.5,
  presence_penalty=0.5
)

informal_summary = response_informal.choices[0].text.strip().replace("Informal Summary:", "").strip()

print("Informal Summary:\n", informal_summary)

# Save the responses to a text file
with open("ContentSummarization.txt", "w") as file:
    file.write("Formal Summary:\n" + formal_summary + "\n\n")
    file.write("Informal Summary:\n" + informal_summary + "\n\n")


Formal Summary:
 Once upon a time, in a far away kingdom, lived a beloved princess named Aurora. To celebrate her birthday, the King and Queen threw a grand banquet with two special guests - Flora and Fauna from the magical realm. Flora presented Aurora with an enchanted rosebush that would never lose its petals until true love found her. Fauna then gave her an old book filled with stories of brave knights on daring adventures to save damsels in distress. Inspired by Fauna's words, Aurora vowed to leave home and find true love while holding onto the rosebush as evidence that it will eventually come when meant to be.
Informal Summary:
 Aurora was an adored princess known for her beauty and kindness. For her birthday, the King and Queen held a grand banquet with two magical fairies as guests. Flora presented Aurora with an enchanted rosebush that would never lose its petals until she found true love. Fauna then gave Aurora a book of brave knights going on adventures to save damsels in di